In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
import geopandas as geo
import folium
import random

from datetime import date

import numpy as np

import fiona

import pyproj

import math

In [4]:
#used https://www.geoapify.com/tools/geocoding-online to geocode addresses

properties = pd.read_csv(os.getcwd()+'/cleaned/Property_wAddress.csv')
#properties.drop(columns = 'Unnamed: 14')
properties

,FULLADDRESS,NUMBER,STREET,ZIP,CITY,BORO,BLOCK,LOT,BBL,BIN,PROJECTNAME,lat,lon
0,10 Forrest Street Brooklyn NY 11206,10,Forrest St,11206.0,NY,Brooklyn,3145.0,100,3.031450e+09,3387594.0,Renaissance Estates,40.702520,-73.932573
1,104 Grove Street Brooklyn NY 11221,104,Grove St,11221.0,NY,Brooklyn,3323.0,11,3.033230e+09,3076081.0,Casa Pasiva,40.693131,-73.920585
2,11 DeSales Place Brooklyn NY 11207,11,DeSales Pl,11207.0,NY,Brooklyn,3468.0,50,3.034680e+09,3326478.0,Our Lady of Lourdes,40.680957,-73.906106
3,110 Grove Street Brooklyn NY 11221,110,Grove St,11221.0,NY,Brooklyn,3323.0,14,3.033230e+09,3076082.0,Casa Pasiva,40.693301,-73.920408
4,"111 Linden Street, Brooklyn, NY 11221",109-111,Linden Street,11221.0,NY,Brooklyn,3323.0,62,NaN,3076111.0,RiseBoro TPT X,40.693226,-73.919853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,93-95 Stockholm Street Brooklyn NY 11221,93-95,Stockholm St,11221.0,NY,Brooklyn,3243.0,48,3.032430e+09,3073774.0,Casa Pasiva,40.697469,-73.925110
181,97 Central Avenue Brooklyn NY 11206,97,Central Ave,11206.0,NY,Brooklyn,3173.0,3,3.031730e+09,3072265.0,West Bushwick 203K,40.700577,-73.929174
182,99 Central Ave Brooklyn NY 11206,99,Central Ave,11206.0,NY,Brooklyn,3173.0,1,3.031730e+09,3072263.0,RBHA,40.700542,-73.929297
183,99-105 Central Avenue Brooklyn NY 11206,105,Central Ave,11206.0,NY,Brooklyn,3173.0,1,3.031730e+09,3072263.0,RBHA,40.700520,-73.929114


In [5]:
#used https://www.geoapify.com/tools/geocoding-online to geocode addresses

programs = pd.read_csv(os.getcwd()+'/cleaned/Programs_wAddresses.csv')

cat_colors = {'Empowerment': 'green',
             'Education': 'lightgray',
             'Seniors': 'purple',
             'Housing (Supportive Services)': 'orange',
             'Health': 'red'}

programs

,Category,Program Name,Program,Address,Referral/,lat,lon
0,Empowerment,Steps From Home,Steps From Home (formerly STEHP) provides prev...,"90 Beaver St, Brooklyn, NY 11206",Call: Mon-Fri 9-5; or e-mail,40.699820,-73.936800
1,Empowerment,Home4Good,Empowerment’s program working directly with G...,"90 Beaver St., Brooklyn, NY 11206",Call: Mon-Fri 9-5; or e-mail,40.699820,-73.936800
2,Empowerment,Food Support Connections,"Assistance applying for SNAP benefits, recerti...","184 Wyckoff Ave. Brooklyn, 11237","Call, walk in or e-mail",40.701780,-73.915600
3,Empowerment,Legal Services,"Tenant organizing, landlord/tenant mediation, ...","7 De Sales Place, Brooklyn, NY 11207",Call (Entrance on 1875 Broadway),40.681118,73.906002
4,Empowerment,"Economic Empowerment (Career Coaching, Financi...",•Empowerment’s workforce development program c...,"90 Beaver St. Brooklyn, NY 11206",Email/Call Mon-Fri 9-5,40.699820,-73.936800
5,Empowerment,HomeBase Bushwick,Homebase provides comprehensive outreach and a...,"380 Grove St. Brooklyn, NY 11237",Call: Mon-Fri 9-5; or e-mail/text,40.700270,-73.913500
6,Empowerment,HomeBase Bushwick,Homebase provides comprehensive outreach and a...,"90 Beaver St. Brooklyn, NY 11206",Call: Mon-Fri 9-5; or e-mail/text,40.699820,-73.936800
7,Empowerment,HomeBase Brownsville,Brownsville and Bushwick Homebase each provide...,"7 De Sales Pl, Brooklyn NY 11207",Call: Mon-Fri 9-5; or e-mail/text,40.681118,73.906002
8,Empowerment,HomeBase Brownsville,Brownsville and Bushwick Homebase each provide...,"145 E 98th St, Brooklyn NY 11212",Call: Mon-Fri 9-5; or e-mail/text,40.663733,-73.921117
9,Empowerment,Wellness Raising,Wellness Rising provides residents the tools a...,"518 Bushwick, Brooklyn NY 11206 (couldn't find...",Email/Call,40.704310,-73.939900


In [6]:
#jitter locations of programs at the same address (probably a beter way of addressing this)
programs['lat_new'] = 0.0
programs['lon_new'] = 0.0

for i in range(len(programs)):
    if np.round(programs.at[i, 'lat'], 5) in programs['lat_new'].round(5).values:
        print(programs.at[i, 'Program Name'])
        programs.at[i, 'lat_new'] =  programs.loc[i]['lat']+random.uniform(-0.0001, 0.0001)
        programs.at[i, 'lon_new'] =  programs.loc[i]['lon']+random.uniform(-0.0001, 0.0001)
    else:
        programs.at[i, 'lat_new'] = programs.at[i, 'lat']
        programs.at[i, 'lon_new'] = programs.at[i, 'lon']

programs

Home4Good
Economic Empowerment (Career Coaching, Financial Coaching, Career Validation)
HomeBase Bushwick
HomeBase Brownsville 
Career Preparation (Youth Center)
Beacon Program at  I. S. 562
PEAK (formerly Project Ex.I.T.O.)
My Voice Theater
Meals on Wheels (RiseBoro Home-delivered Meals)


,Category,Program Name,Program,Address,Referral/,lat,lon,lat_new,lon_new
0,Empowerment,Steps From Home,Steps From Home (formerly STEHP) provides prev...,"90 Beaver St, Brooklyn, NY 11206",Call: Mon-Fri 9-5; or e-mail,40.699820,-73.936800,40.699820,-73.936800
1,Empowerment,Home4Good,Empowerment’s program working directly with G...,"90 Beaver St., Brooklyn, NY 11206",Call: Mon-Fri 9-5; or e-mail,40.699820,-73.936800,40.699919,-73.936811
2,Empowerment,Food Support Connections,"Assistance applying for SNAP benefits, recerti...","184 Wyckoff Ave. Brooklyn, 11237","Call, walk in or e-mail",40.701780,-73.915600,40.701780,-73.915600
3,Empowerment,Legal Services,"Tenant organizing, landlord/tenant mediation, ...","7 De Sales Place, Brooklyn, NY 11207",Call (Entrance on 1875 Broadway),40.681118,73.906002,40.681118,73.906002
4,Empowerment,"Economic Empowerment (Career Coaching, Financi...",•Empowerment’s workforce development program c...,"90 Beaver St. Brooklyn, NY 11206",Email/Call Mon-Fri 9-5,40.699820,-73.936800,40.699840,-73.936797
5,Empowerment,HomeBase Bushwick,Homebase provides comprehensive outreach and a...,"380 Grove St. Brooklyn, NY 11237",Call: Mon-Fri 9-5; or e-mail/text,40.700270,-73.913500,40.700270,-73.913500
6,Empowerment,HomeBase Bushwick,Homebase provides comprehensive outreach and a...,"90 Beaver St. Brooklyn, NY 11206",Call: Mon-Fri 9-5; or e-mail/text,40.699820,-73.936800,40.699747,-73.936775
7,Empowerment,HomeBase Brownsville,Brownsville and Bushwick Homebase each provide...,"7 De Sales Pl, Brooklyn NY 11207",Call: Mon-Fri 9-5; or e-mail/text,40.681118,73.906002,40.681113,73.905908
8,Empowerment,HomeBase Brownsville,Brownsville and Bushwick Homebase each provide...,"145 E 98th St, Brooklyn NY 11212",Call: Mon-Fri 9-5; or e-mail/text,40.663733,-73.921117,40.663733,-73.921117
9,Empowerment,Wellness Raising,Wellness Rising provides residents the tools a...,"518 Bushwick, Brooklyn NY 11206 (couldn't find...",Email/Call,40.704310,-73.939900,40.704310,-73.939900


In [7]:
programs.to_csv(os.getcwd()+'/cleaned/Programs_wAddressesJitter.csv', index=False)

Geography data from https://www.nyc.gov/site/planning/data-maps/open-data/census-download-metadata.page and svi from: https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html

In [8]:
gdf = geo.read_file(os.getcwd()+'\\nyct2020_23b\\nyct2020_23b\\nyct2020.shp')
gdf = gdf.astype({'GEOID': 'str'})
gdf

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,GEOID,PUMA,Shape_Leng,Shape_Area,geometry
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,4121,10833.043929,1.843005e+06,"MULTIPOLYGON (((972081.788 190733.467, 972184...."
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4103,4754.495247,9.723121e+05,"POLYGON ((988548.218 197770.375, 987978.808 19..."
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,4103,6976.286215,2.582705e+06,"POLYGON ((986961.185 199553.643, 987206.139 19..."
3,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,4103,5075.332000,1.006117e+06,"POLYGON ((987475.016 200297.218, 987705.443 20..."
4,14.02,1,Manhattan,001402,1001402,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4103,4459.156019,1.226206e+06,"POLYGON ((988387.669 201258.312, 988621.002 20..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,666,3,Brooklyn,066600,3066600,None,Marine Park-Plumb Island,BK1891,BK18,BK18 Canarsie-Flatlands (CD 18 Approximation),36047066600,4318,96948.219292,4.327690e+07,"MULTIPOLYGON (((1007822.043 152881.588, 100783..."
2321,24,1,Manhattan,002400,1002400,None,East Village,MN0303,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061002400,4103,9497.307624,1.863567e+06,"MULTIPOLYGON (((993355.892 200980.164, 993519...."
2322,555,3,Brooklyn,055500,3055500,None,Williamsburg,BK0102,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),36047055500,4301,7293.581985,2.830412e+06,"POLYGON ((995613.061 201097.814, 995447.441 20..."
2323,177,3,Brooklyn,017700,3017700,None,Prospect Park,BK5591,BK55,BK55 Prospect Park (JIA 55 Approximation),36047017700,4306,24199.068858,2.732422e+07,"POLYGON ((993269.739 184419.645, 993368.256 18..."


In [9]:
svi = pd.read_csv(os.getcwd()+'/'+'NewYork.csv', usecols = ['STCNTY', 'COUNTY', 'FIPS', 'LOCATION', 'RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4'])
svi = svi.astype({'FIPS': 'str'})
svi = svi.replace(-999,np.NaN)
svi

,STCNTY,COUNTY,FIPS,LOCATION,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4
0,36001,Albany,36001000100,"Census Tract 1, Albany County, New York",0.9384,0.8950,0.7547,0.3587
1,36001,Albany,36001000201,"Census Tract 2.01, Albany County, New York",0.6864,0.6133,0.7662,0.4169
2,36001,Albany,36001000202,"Census Tract 2.02, Albany County, New York",0.8324,0.4087,0.8901,0.4382
3,36001,Albany,36001000301,"Census Tract 3.01, Albany County, New York",0.8466,0.9497,0.7950,0.2789
4,36001,Albany,36001000302,"Census Tract 3.02, Albany County, New York",0.4773,0.1174,0.4939,0.3523
...,...,...,...,...,...,...,...,...
5389,36123,Yates,36123150301,"Census Tract 1503.01, Yates County, New York",0.5892,0.6295,0.1277,0.5250
5390,36123,Yates,36123150302,"Census Tract 1503.02, Yates County, New York",0.5362,0.7016,0.0377,0.2396
5391,36123,Yates,36123150400,"Census Tract 1504, Yates County, New York",0.5129,0.0505,0.0962,0.7095
5392,36123,Yates,36123150501,"Census Tract 1505.01, Yates County, New York",0.4888,0.7267,0.2508,0.3452


In [10]:
geo_df = gdf.merge(svi, left_on='GEOID', right_on = 'FIPS', how='left')
geo_df = geo_df.rename(columns={'RPL_THEME1': 'SVI Socio', 'RPL_THEME2': 'SVI Household', 'RPL_THEME3': 'SVI Race/Ethnicity', 'RPL_THEME4': 'SVI Housing'})
geo_df

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,...,Shape_Area,geometry,STCNTY,COUNTY,FIPS,LOCATION,SVI Socio,SVI Household,SVI Race/Ethnicity,SVI Housing
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),...,1.843005e+06,"MULTIPOLYGON (((972081.788 190733.467, 972184....",36061.0,New York,36061000100,"Census Tract 1, New York County, New York",NaN,NaN,NaN,NaN
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,9.723121e+05,"POLYGON ((988548.218 197770.375, 987978.808 19...",36061.0,New York,36061000201,"Census Tract 2.01, New York County, New York",0.7886,0.9696,0.8622,0.9789
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,2.582705e+06,"POLYGON ((986961.185 199553.643, 987206.139 19...",36061.0,New York,36061000600,"Census Tract 6, New York County, New York",0.9602,0.9340,0.8723,0.9601
3,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,1.006117e+06,"POLYGON ((987475.016 200297.218, 987705.443 20...",36061.0,New York,36061001401,"Census Tract 14.01, New York County, New York",0.0799,0.5272,0.4250,0.9797
4,14.02,1,Manhattan,001402,1001402,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,1.226206e+06,"POLYGON ((988387.669 201258.312, 988621.002 20...",36061.0,New York,36061001402,"Census Tract 14.02, New York County, New York",0.6949,0.8708,0.7258,0.7420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,666,3,Brooklyn,066600,3066600,None,Marine Park-Plumb Island,BK1891,BK18,BK18 Canarsie-Flatlands (CD 18 Approximation),...,4.327690e+07,"MULTIPOLYGON (((1007822.043 152881.588, 100783...",36047.0,Kings,36047066600,"Census Tract 666, Kings County, New York",NaN,NaN,NaN,NaN
2321,24,1,Manhattan,002400,1002400,None,East Village,MN0303,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,1.863567e+06,"MULTIPOLYGON (((993355.892 200980.164, 993519....",36061.0,New York,36061002400,"Census Tract 24, New York County, New York",0.9515,0.9919,0.9344,0.6764
2322,555,3,Brooklyn,055500,3055500,None,Williamsburg,BK0102,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),...,2.830412e+06,"POLYGON ((995613.061 201097.814, 995447.441 20...",36047.0,Kings,36047055500,"Census Tract 555, Kings County, New York",0.1242,0.0193,0.5635,0.6226
2323,177,3,Brooklyn,017700,3017700,None,Prospect Park,BK5591,BK55,BK55 Prospect Park (JIA 55 Approximation),...,2.732422e+07,"POLYGON ((993269.739 184419.645, 993368.256 18...",36047.0,Kings,36047017700,"Census Tract 177, Kings County, New York",NaN,NaN,NaN,NaN


In [13]:
#create base map
m = folium.Map(location=[40.67762837367161, -73.9434755141601], zoom_start=10 , tiles=None, zoom_control = False)
base_map = folium.FeatureGroup(name='Basemap', overlay=True)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

#make label info
toolTip = {}
toolTip['SVI Socio'] = ['NTAName', 'SVI Socio', 'SVI Household', 'SVI Race/Ethnicity', 'SVI Housing']
toolTip['SVI Household'] = ['NTAName', 'SVI Socio', 'SVI Household', 'SVI Race/Ethnicity', 'SVI Housing']
toolTip['SVI Race/Ethnicity'] = ['NTAName', 'SVI Socio', 'SVI Household', 'SVI Race/Ethnicity', 'SVI Housing']
toolTip['SVI Housing'] = ['NTAName', 'SVI Socio', 'SVI Household', 'SVI Race/Ethnicity', 'SVI Housing']
    
#make layers
layers = ['SVI Socio', 'SVI Household', 'SVI Race/Ethnicity', 'SVI Housing']
for l in layers:
    print(l)
    # plot chorpleth over the base map
    cp = folium.Choropleth(geo_df,                                # geo data
                      data=geo_df,                           # data
                      key_on='feature.properties.NTAName',
                      columns=['NTAName', l],
                      fill_color='RdPu',                     # cmap
                      line_weight=0.1,                       # line wight (of the border)
                      line_opacity=0.5,                      # line opacity (of the border)
                      overlay = False,
                      legend_name = l,
                      name =  l
                      ).add_to(m)    # name on the legend color bar
    folium.GeoJsonPopup(toolTip[l]).add_to(cp.geojson) #add information for each geoid

#add property points
locations = properties[['lat', 'lon']]
locationlist = locations.values.tolist()
for point in range(0, len(locationlist)):
    if not math.isnan(locationlist[point][0]) and not math.isnan(locationlist[point][1]):
        folium.Marker(locationlist[point], popup=str(properties['PROJECTNAME'][point])+ ' ('+properties['FULLADDRESS'][point]+')').add_to(m)
        
#add program points
locations = programs[['lat_new', 'lon_new']]
locationlist = locations.values.tolist()
for point in range(0, len(locationlist)):
    if not math.isnan(locationlist[point][0]) and not math.isnan(locationlist[point][1]):
        folium.Marker(locationlist[point], 
                      popup=str(programs['Program Name'][point])+ ' ('+programs['Address'][point]+')',
                     icon=folium.Icon(color=cat_colors[programs['Category'][point]])).add_to(m)

# add layer controls
folium.LayerControl().add_to(m)

#m

SVI Socio
SVI Household
SVI Race/Ethnicity
SVI Housing


In [14]:
m.save('mapExample.html')